In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize
import folium

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'SAKBIYDVVZEYCJCYQ4W0NEFXP2MBOWQW1L1HC5LIX3MCM0YD' #Foursquare ID
CLIENT_SECRET = 'W0E5WOGW0XOCF2ZGE2LS4S4MUXRFJRPEAN31YIZLJXTE3Y3R' #Foursquare Secret
VERSION = '20180605' #Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: SAKBIYDVVZEYCJCYQ4W0NEFXP2MBOWQW1L1HC5LIX3MCM0YD


In [3]:
LIMIT = 100 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1e0931735") #COFFEE SHOP CATEGORY
    results[city] = requests.get(url).json()

In [6]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of coffee shops in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of coffee shops in New York, NY =  238
Showing Top 100
Total number of coffee shops in Chicago, IL =  186
Showing Top 100
Total number of coffee shops in San Francisco, CA =  225
Showing Top 100
Total number of coffee shops in Jersey City, NJ =  106
Showing Top 100
Total number of coffee shops in Boston, MA =  188
Showing Top 100


In [12]:
maps[cities[0]]

maps[cities[1]]

In [9]:
maps[cities[2]]

In [10]:
maps[cities[3]]

In [11]:
maps[cities[4]]

Based on the maps, we can see that New York and Jersey City are the most dense cities with coffee shops. Alos, they are just shore away and which are divided by Hudson river.

Let's have a look at the density.

For this I will use some basic statistics. I will get the mean location of the coffee sahops which should be near to most of them if they are really dense or far if not.

Next I will take the average of the distance of the venues to the mean coordinates.

In [13]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.023694082665728793
Chicago, IL
Mean Distance from Mean coordinates
0.05165194944620294
San Francisco, CA
Mean Distance from Mean coordinates
0.026080058112082964
Jersey City, NJ
Mean Distance from Mean coordinates
0.017477284000931233
Boston, MA
Mean Distance from Mean coordinates
0.031488933521412917


In [14]:
maps[cities[0]]

In [15]:
maps[cities[1]]

In [16]:
maps[cities[2]]

In [17]:
maps[cities[3]]

In [18]:
maps[cities[4]]

## Based on the Mean Distance from Mean Coordinates of the coffee shops, Jersey City, New York, San Fransisco, Boston, Chicago are the best places to surrounded by coffee shops respectively. jersey City is the best place for a person to surrounded by coffee shops from the mean location. and as an add-on New York is the second best place and which is just a shore away.

1. Jersey City,  NJ
2. New York, NY
3. San Fransisco, CA
4. Boston, MA
5. Chicago, IL

So based on my analysis, I conclude that our tourist's best interest would be arrive at the mean coordinate to surround himself with the 100 coffee shops there!!